# Análise de similaridade com SBERT + CodeBERT
Este notebook usa um algoritmo com **SBERT** + **CodeBERT** para análise de similaridade (cosine), indicando o quão semanticamente semelhantes dois scripts são - representados como vetores - com base na similaridade do cosseno entre eles.

In [ ]:
# --------------------------------------------------
# Instalar dependências
# --------------------------------------------------

!pip install -q sentence-transformers openpyxl pandas

In [2]:
# --------------------------------------------------
# Algoritmo geral
# --------------------------------------------------

# Importar bibliotecas
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# Carregar a planilha de execuções
#df = pd.read_excel("resultados_obda_cb1.xlsx")
df = pd.read_excel("resultados_obda_cb2.xlsx")

df.columns = df.columns.str.strip()

# Scripts de referência (CB1)
#ref_sql = "SELECT animals.id FROM animals JOIN speciesList ON animals.taxon = speciesList.scientificName WHERE speciesList.class = 'Giraffe';"
#ref_map = "mappingId	M1 target		:Animal/{id} a :Giraffe .  source		SELECT animals.id FROM animals JOIN speciesList ON animals.taxon = speciesList.scientificName WHERE speciesList.class = 'Giraffe'"
#ref_sparql = "PREFIX : <http://example.org/ontology#> SELECT ?animal WHERE { ?animal a :Giraffe . }"

# Scripts de referência (CB2)
ref_sql = "SELECT animals.id, animals.taxon FROM animals JOIN speciesList ON animals.taxon = speciesList.scientificName WHERE speciesList.class = 'Giraffe';"
ref_map = "mappingId	M1 target		:Animal/{id} a :Giraffe ; :has-taxon {taxon} . source		SELECT animals.id, animals.taxon FROM animals JOIN speciesList ON animals.taxon = speciesList.scientificName WHERE speciesList.class = 'Giraffe'"
ref_sparql = "PREFIX : <http://example.org/ontology#> SELECT ?animal ?taxon WHERE { ?animal a :Giraffe ; :has-taxon ?taxon . }"

# Carregar modelo
model = SentenceTransformer("microsoft/codebert-base")

# Função de similaridade
def sim(texto, referencia):
    if isinstance(texto, str) and texto.strip():
        emb1 = model.encode(texto, convert_to_tensor=True)
        emb2 = model.encode(referencia, convert_to_tensor=True)
        return round(util.cos_sim(emb1, emb2).item(), 4)
    return ""

# Aplicar cálculo
df["Sim_SQL"] = df["SQL (Gerado)"].apply(lambda x: sim(x, ref_sql))
df["Sim_Mapping"] = df["Mapping (Gerado)"].apply(lambda x: sim(x, ref_map))
df["Sim_SPARQL"] = df["SPARQL (Gerado)"].apply(lambda x: sim(x, ref_sparql))

# Salvar resultado
output_name = "planilha_similaridade_OBDA_preenchida.xlsx"
df.to_excel(output_name, index=False)
